# Politician Activity on Facebook by Political Affiliation

The parameters in the cell below can be adjusted to explore other political affiliations and time frames.

### How to explore other political affiliation?
The ***affiliation*** parameter can be use to aggregate politicians by their political affiliations. The column `affiliation` in this [this other notebook](../politicians.ipynb?autorun=true) show the politicians that belong each political affiliation.

***Alternatively***, you can direcly use the [politicians API](http://mediamonitoring.gesis.org/api/politicians/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

In [ ]:
# Parameters: 
affiliation <- "CDU"
aggregation <- "week" # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

## Using API

In [ ]:
base <- "193.175.238.88/api/politicians/"
call1 <- paste(base,"all/", sep="")
get_politicians_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
party_df <- get_politicians_df[get_politicians_df$affiliation == affiliation & !is.null(get_politicians_df$fb_ids) ,]
posts <- data.frame()
comments <- data.frame()
for (fb_id in party_df$fb_ids) {
    for (fb in fb_id) {
        call_posts <- paste(base,"facebook/posts_by/politicians/user_id/", toString(fb), "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
        call_comments <- paste(base,"facebook/comments_by/politicians/user_id/", toString(fb), "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
        if (length(fromJSON(content(GET(call_posts), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
            posts_unique <- as.data.frame(fromJSON(content(GET(call_posts), "text", encoding="UTF-8"), flatten = TRUE))
            posts_unique$fb_user_id = fb
            posts <- rbind(posts, posts_unique)
            }
        if (length(fromJSON(content(GET(call_comments), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
            comments_unique <- as.data.frame(fromJSON(content(GET(call_comments), "text", encoding="UTF-8"), flatten = TRUE))
            comments_unique$fb_user_id = fb
            comments <- rbind(comments, comments_unique)
            }
    }
}

posts <- summarise(group_by(posts, labels), posts = sum(values), shares = sum(shares), replies = sum(replies), reactions = sum(reactions), likes = sum(likes), response_type = aggregation)
comments <- summarise(group_by(comments, labels), comments = sum(values), replies = sum(replies), likes = sum(likes), response_type = aggregation)
merged <- merge(posts, comments, by='labels')

In [ ]:
#plotting facebook posts
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
    geom_line(aes(y = posts, color="Posts", group=response_type.x)) +
    geom_line(aes(y = replies.x, color="Replies", group=response_type.x)) +
    geom_line(aes(y = shares, color="Shares", group=response_type.x)) +
    geom_line(aes(y = reactions, color="Reactions", group=response_type.x)) +
    geom_line(aes(y = likes.x, color="Likes", group=response_type.x,)) +
    labs(title = "Facebook (Post Activity)", x = "", y = "Posts") +
    theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), plot.title = element_text(size=10)) +
    scale_x_date(date_breaks = "1 month") + scale_y_continuous(labels = scales::number_format(big.mark = "")) 

In [ ]:
#plotting facebook comments
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = comments, color="Comments", group=response_type.y)) +
  geom_line(aes(y = replies.y, color="Replies", group=response_type.y)) +
  geom_line(aes(y = likes.y, color="Likes", group=response_type.x)) +
  labs(title = "Facebook (Comments Activity)", x = "", y = "Comments") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), plot.title = element_text(size=10)) +
  scale_x_date(date_breaks = "1 month") + scale_y_continuous(labels = scales::number_format(big.mark = ""))